# DATA SCIENCE & AI (FIAP MBA) - Sabrina Otoni da Silva - 2023

## Projeto nº 6 (NÃO FINALIZADO AINDA!)

### Professor: Ahirton Lopes (Machine Learning: Aula 3) -- https://github.com/ahirtonlopes/Mastering-Machine-Learning

#### KNN para prever rotatividade de clientes com base nas características de cada conta (churn prediction)

O foco é apenas entender a aplicação do modelo. Para esta demo utilizaremos um problema clássico em ML da indústria de telecomunicações (telco's).

Ref: https://www.kaggle.com/datasets/mnassrib/telecom-churn-datasets

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer, MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [2]:
data = pd.read_csv('Orange_Telco.csv', sep=',')

In [3]:
data.head()

,state,account_length,area_code,phone_number,intl_plan,voice_mail_plan,number_vmail_messages,total_day_minutes,total_day_calls,total_day_charge,...,total_eve_calls,total_eve_charge,total_night_minutes,total_night_calls,total_night_charge,total_intl_minutes,total_intl_calls,total_intl_charge,number_customer_service_calls,churned
0,KS,128,415,382-4657,no,yes,25,265.1,110,45.07,...,99,16.78,244.7,91,11.01,10.0,3,2.70,1,False
1,OH,107,415,371-7191,no,yes,26,161.6,123,27.47,...,103,16.62,254.4,103,11.45,13.7,3,3.70,1,False
2,NJ,137,415,358-1921,no,no,0,243.4,114,41.38,...,110,10.30,162.6,104,7.32,12.2,5,3.29,0,False
3,OH,84,408,375-9999,yes,no,0,299.4,71,50.90,...,88,5.26,196.9,89,8.86,6.6,7,1.78,2,False
4,OK,75,415,330-6626,yes,no,0,166.7,113,28.34,...,122,12.61,186.9,121,8.41,10.1,3,2.73,3,False


Os dados contêm um estado, código de área (zip code) e número de telefone. Você acha que esses são recursos bons o suficiente a serem usados para a criação de um modelo de aprendizagem de máquina? Por que sim ou por que não? Neste exemplo, nós não os utilizaremos, logo, descartaremos esses dados.

In [4]:
data.drop(['state', 'area_code', 'phone_number'], axis=1, inplace=True)

In [5]:
data.columns

Index(['account_length', 'intl_plan', 'voice_mail_plan',
       'number_vmail_messages', 'total_day_minutes', 'total_day_calls',
       'total_day_charge', 'total_eve_minutes', 'total_eve_calls',
       'total_eve_charge', 'total_night_minutes', 'total_night_calls',
       'total_night_charge', 'total_intl_minutes', 'total_intl_calls',
       'total_intl_charge', 'number_customer_service_calls', 'churned'],
      dtype='object')

## Pré Processamento

* Algumas das colunas são dados categóricos e outras são floats. Esses recursos precisarão ser codificados numericamente;

* Finalmente, lembre-se que para aplicação correta do alg. KNN os dados devem estar em escala;

* Logo, escalaremos os dados usando um dos métodos de dimensionamento existentes no Scikitlearn;

* Estaremos, primeiramente, binarizando as colunas 'intl_plan', 'voice_mail_plan' e 'churned'.

Ref. https://scikit-learn.org/stable/modules/preprocessing.html

In [6]:
lb = LabelBinarizer()

for col in ['intl_plan', 'voice_mail_plan', 'churned']:
    data[col] = lb.fit_transform(data[col])

In [7]:
import warnings
warnings.filterwarnings('ignore', module='sklearn')

In [8]:
msc = MinMaxScaler()

data = pd.DataFrame(msc.fit_transform(data), columns=data.columns)

In [9]:
data.head()

,account_length,intl_plan,voice_mail_plan,number_vmail_messages,total_day_minutes,total_day_calls,total_day_charge,total_eve_minutes,total_eve_calls,total_eve_charge,total_night_minutes,total_night_calls,total_night_charge,total_intl_minutes,total_intl_calls,total_intl_charge,number_customer_service_calls,churned
0,0.524793,0.0,1.0,0.480769,0.754196,0.666667,0.754183,0.542755,0.582353,0.542866,0.619494,0.520000,0.619584,0.500,0.15,0.500000,0.111111,0.0
1,0.438017,0.0,1.0,0.500000,0.459744,0.745455,0.459672,0.537531,0.605882,0.537690,0.644051,0.588571,0.644344,0.685,0.15,0.685185,0.111111,0.0
2,0.561983,0.0,0.0,0.000000,0.692461,0.690909,0.692436,0.333242,0.647059,0.333225,0.411646,0.594286,0.411930,0.610,0.25,0.609259,0.000000,0.0
3,0.342975,1.0,0.0,0.000000,0.851778,0.430303,0.851740,0.170195,0.517647,0.170171,0.498481,0.508571,0.498593,0.330,0.35,0.329630,0.222222,0.0
4,0.305785,1.0,0.0,0.000000,0.474253,0.684848,0.474230,0.407754,0.717647,0.407959,0.473165,0.691429,0.473270,0.505,0.15,0.505556,0.333333,0.0


## Ajuste do modelo

* Agora vamos separar as colunas de recursos (ou seja, tudo exceto nosso "alvo" 'churned') do rótulo ('churned');

* Estaremos criando duas tabelas;

* Ajustaremos um modelo de K vizinhos mais próximos com um valor de 'k = 3' a esses dados e vamos predizer o resultado nos mesmos dados.

[Observação] Normalmente os valores de K tendem a ser ímpares. O k, no algoritmo KNN, representa o número de vizinhos mais próximos com os quais estamos comparando um determinado ponto de dados. Portanto, não importa se você tem 2 ou n classes, se você escolhe um mesmo k, há um risco de empate na decisão de qual classe você deve definir uma nova instância.

In [10]:
x_cols = [x for x in data.columns if x != 'churned']

X_data = data[x_cols]
y_data = data['churned']

In [11]:
knn = KNeighborsClassifier(n_neighbors=3)

knn = knn.fit(X_data, y_data)

y_pred = knn.predict(X_data.values)

## Verificação quanto a Acurácia/Matriz de Confusão I

A Acurácia por si só é simplesmente a porcentagem de rótulos que foram previstos corretamente (verdadeiros ou falsos).

* Vamos então escrever uma função para calcular a precisão usando os rótulos reais e preditos;

* Usando a função, vamos calcular a precisão desse modelo de K vizinho mais próximos em nossos dados.

In [12]:
#Função para calcular a % de valores previstos corretamente
def accuracy(real, predict):
    return sum(y_data == y_pred) / float(real.shape[0])

In [13]:
print(confusion_matrix(y_data, y_pred))

[[4263   30]
 [ 259  448]]


In [14]:
print(classification_report(y_data,y_pred))

              precision    recall  f1-score   support

         0.0       0.94      0.99      0.97      4293
         1.0       0.94      0.63      0.76       707

    accuracy                           0.94      5000
   macro avg       0.94      0.81      0.86      5000
weighted avg       0.94      0.94      0.94      5000



In [15]:
print(accuracy_score(y_data, y_pred))

0.9422


## Ajuste do Modelo II

No exemplo da célula anterior acabamos nos deparando com um erro muito comum, qual foi? acabamos predizendo e treinando e testando nosso modelo nos mesmos dados.

Vamos arrumar!


In [16]:
X = data.iloc[:,[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]].values
y = data.iloc[:,17].values

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .3, random_state=25)

In [18]:
knn = KNeighborsClassifier(n_neighbors=3)

knn = knn.fit(X_train, y_train)

pred = knn.predict(X_test)

# Verificação quanto a Acurácia/Matriz de Confusão II

In [19]:
print(confusion_matrix(y_test,pred))

[[1262   29]
 [ 114   95]]


In [20]:
print(classification_report(y_test,pred))

              precision    recall  f1-score   support

         0.0       0.92      0.98      0.95      1291
         1.0       0.77      0.45      0.57       209

    accuracy                           0.90      1500
   macro avg       0.84      0.72      0.76      1500
weighted avg       0.90      0.90      0.89      1500



In [21]:
print(accuracy_score(y_test, pred))

0.9046666666666666


*[Desafio]*

### Nas células a seguir, tente melhorar os resultados alcançados, observando a métrica F1 score.

Alguns processos e técnicas que podem ser utilizados:

- Utilização de técnicas de oversampling para lidar com o dataset desabalanceado (ref: https://machinelearningmastery.com/smote-oversampling-for-imbalanced-classification/ // ref: https://machinelearningmastery.com/tactics-to-combat-imbalanced-classes-in-your-machine-learning-dataset);

(Dica: bases desbalanceadas são muito comuns em problemas de detecção de anomalias! - ref: https://www.analyticsvidhya.com/blog/2017/03/imbalanced-data-classification/);

- Utilização de técnicas de engenharia de atributos, verificação de outliers, one-hot-encoding (ref: https://towardsdatascience.com/feature-engineering-for-machine-learning-3a5e293a5114);

- Redução de dimensionalidade de atributos (ref: https://towardsdatascience.com/all-you-need-to-know-about-pca-technique-in-machine-learning-443b0c2be9a1).